In [1]:
%pip install -r requirements.txt 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 114.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 123.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 47.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 104.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 108.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 138.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 26.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 944.1/944.1 kB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')


In [3]:
%run -i data_generation/simple_linear_regression.py


### Training helper function

In [4]:
def make_train_step(model,loss_fn,optimizer):

    def perform_train_step(x,y):
        # set to train mode
        model.train()

        yhat = model(x)

        loss = loss_fn(yhat,y)
        #backprop
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()

        return loss.item()
    
    return perform_train_step

In [5]:
%run -i data_preparation/v0.py

Model config v1

In [6]:
%%writefile model_configuration/v1.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'

lr = 0.1

torch.manual_seed(42)

model = nn.Sequential(nn.Linear(1,1)).to(device)

optimizer = optim.SGD(model.parameters(),lr = lr)

loss_fn = nn.MSELoss(reduction='mean')

train_step = make_train_step(model,loss_fn,optimizer)


Overwriting model_configuration/v1.py


In [7]:
%run -i model_configuration/v1.py

In [8]:
train_step

<function __main__.make_train_step.<locals>.perform_train_step(x, y)>

Rewrite model_training

In [9]:
%%writefile model_training/v1.py

n_epochs = 1000

losses = []

for epoch in range(n_epochs):

    loss = train_step(x_train_tensor,y_train_tensor)
    losses.append(loss)

Overwriting model_training/v1.py


In [10]:
%run -i model_training/v1.py

In [11]:
model.state_dict()

OrderedDict([('0.weight', tensor([[1.9690]])), ('0.bias', tensor([1.0235]))])

### Datasets

In [17]:
class CustomDataset(Dataset):

    def __init__(self,x_tensor,y_tensor):
        self.x = x_tensor
        self.y = y_tensor
    
    def __getitem__(self, index):
        return (self.x[index],self.y[index])
    
    def __len__(self):
        return len(self.x)

In [18]:
x_train_tensor = torch.as_tensor(x_train).float()
y_train_tensor = torch.as_tensor(y_train).float()

train_data = CustomDataset(x_train_tensor,y_train_tensor)
print(train_data[0])

(tensor([0.7713]), tensor([2.4745]))


In [20]:
# TensorDataset class

train_data = TensorDataset(x_train_tensor,y_train_tensor)
print(train_data[0])

(tensor([0.7713]), tensor([2.4745]))


### DataLoader

In [21]:
train_loader = DataLoader(dataset = train_data,batch_size=16,shuffle=True)

In [22]:
next(iter(train_loader))

[tensor([[0.2809],
         [0.3253],
         [0.1560],
         [0.5924],
         [0.0651],
         [0.8872],
         [0.4938],
         [0.0055],
         [0.1409],
         [0.0885],
         [0.1849],
         [0.7290],
         [0.8662],
         [0.3117],
         [0.6842],
         [0.1987]]),
 tensor([[1.5846],
         [1.8057],
         [1.2901],
         [2.1687],
         [1.1559],
         [2.8708],
         [1.9060],
         [1.0632],
         [1.1211],
         [1.0708],
         [1.5888],
         [2.4927],
         [2.6805],
         [1.7637],
         [2.3492],
         [1.2654]])]

Refactor the training procedure



In [23]:
%%writefile data_preparation/v1.py

x_train_tensor = torch.as_tensor(x_train).float()
y_train_tensor = torch.as_tensor(y_train).float()

train_data = TensorDataset(x_train_tensor,y_train_tensor)

train_loader = DataLoader(train_data,batch_size = 16,shuffle=True)  

Writing data_preparation/v1.py


In [24]:
%run -i data_preparation/v1.py

In [25]:
%run -i model_configuration/v1.py

In [26]:
%%writefile model_training/v2.py

n_epochs = 1000

losses = []

for epoch in range(n_epochs):

    mini_batch_losses = []
    for x_batch,y_batch in train_loader:

        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        mini_batch_loss = train_step(x_batch,y_batch)
        mini_batch_losses.append(mini_batch_loss)

    loss = np.mean(mini_batch_losses)
    losses.append(loss)

Writing model_training/v2.py


In [27]:
%run -i model_training/v2.py

In [28]:
model.state_dict()

OrderedDict([('0.weight', tensor([[1.9696]])), ('0.bias', tensor([1.0243]))])

In [29]:
def mini_batch(device,data_loader,step):
    mini_batch_losses = []
    for x_batch,y_batch in data_loader:

        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        mini_batch_loss = step(x_batch,y_batch)
        mini_batch_losses.append(mini_batch_loss)

    loss = np.mean(mini_batch_losses)
    return loss

In [30]:
%%writefile model_training/v3.py

 # Defines number of epochs
n_epochs = 200

losses = []

for epoch in range(n_epochs):
 # inner loop
    loss = mini_batch(device, train_loader, train_step)
    losses.append(loss)

Writing model_training/v3.py


In [31]:
%run -i model_training/v3.py

In [32]:
model.state_dict()

OrderedDict([('0.weight', tensor([[1.9696]])), ('0.bias', tensor([1.0260]))])

### Random split

In [33]:
%%writefile data_preparation/v2.py

from idna import valid_label_length


torch.manual_seed(42)

x_tensor = torch.as_tensor(x).float()
y_tensor = torch.as_tensor(y).float()

dataset = TensorDataset(x_tensor,y_tensor)

ratio = 0.8
n_total = len(dataset)
n_train = int(ratio * n_total)
n_val = n_total - n_train

train_data,val_data = random_split(dataset,[n_train,n_val])

train_loader = DataLoader(dataset = train_data,batch_size = 16, shuffle = True)

val_loader = DataLoader(dataset = val_data,batch_size = 16)

Writing data_preparation/v2.py


In [34]:
%run -i data_preparation/v2.py

### Evaluation

In [35]:
def make_val_step(model,loss_fn):

    def perform_val_step(x,y):

        model.eval()

        yhat = model(x)
        loss = loss_fn(yhat,y)

        return loss.item()
    
    return perform_val_step

In [36]:
%%writefile model_configuration/v2.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'

lr = 0.1

torch.manual_seed(42)

model = nn.Sequential(nn.Linear(1,1)).to(device)

optimizer = optim.SGD(model.parameters(),lr = lr)

loss_fn = nn.MSELoss(reduction = 'mean')

train_step = make_train_step(model,loss_fn,optimizer)

val_step = make_val_step(model,loss_fn)

Writing model_configuration/v2.py


In [37]:
%run -i model_configuration/v2.py

In [38]:
%%writefile model_training/v4.py

n_epochs = 200

losses = []
val_losses = []

for epoch in range(n_epochs):

    loss = mini_batch(device,train_loader,train_step)
    losses.append(loss)

    with torch.no_grad():
        val_loss = mini_batch(device,val_loader,val_step)
        val_losses.append(val_loss)

Writing model_training/v4.py


In [39]:
%run -i model_training/v4.py

In [40]:
model.state_dict()

OrderedDict([('0.weight', tensor([[1.9599]])), ('0.bias', tensor([1.0151]))])

### Plotting losses using Tensorboard

In [41]:
%load_ext tensorboard

In [46]:
%tensorboard --logdir runs

Reusing TensorBoard on port 6007 (pid 2819), started 0:05:20 ago. (Use '!kill 2819' to kill it.)

In [43]:
writer = SummaryWriter('runs/test')

In [44]:
writer.add_graph(model)

TypeError: 'NoneType' object is not iterable

In [45]:
dummy_x, dummy_y = next(iter(train_loader))
writer.add_graph(model, dummy_x.to(device))

In [47]:
writer.add_scalars(main_tag='loss',
tag_scalar_dict={'training':loss,
                'validation':val_loss},
                global_step=epoch)

In [48]:
%run -i data_preparation/v2.py

In [49]:
%%writefile model_configuration/v3.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'

lr = 0.1

torch.manual_seed(42)

model = nn.Sequential(nn.Linear(1,1)).to(device)

optimizer = optim.SGD(model.parameters(),lr=lr)

loss_fn = nn.MSELoss(reduction = 'mean')

train_step = make_train_step(model, loss_fn, optimizer)
val_step = make_val_step(model, loss_fn)

writer = SummaryWriter('runs/simple_linear_regression')

# Fetches a single mini-batch so we can use add_graph
x_dummy, y_dummy = next(iter(train_loader))
writer.add_graph(model, x_dummy.to(device))

Writing model_configuration/v3.py


In [50]:
%run -i model_configuration/v3.py

In [51]:
%%writefile model_training/v5.py

n_epochs = 200

losses = []
val_losses = []

for epoch in range(n_epochs):

    loss = mini_batch(device,train_loader,train_step)
    losses.append(loss)

    with torch.no_grad():
        val_loss = mini_batch(device,val_loader,val_step)
        val_losses.append(val_loss)

    writer.add_scalars(main_tag='loss',
                       tag_scalar_dict=
                       {'training':loss,
                       'validation':val_loss
                       },
                       global_step=epoch)
writer.close()


Writing model_training/v5.py


In [52]:
%run -i model_training/v5.py

In [53]:
model.state_dict()

OrderedDict([('0.weight', tensor([[1.9583]])), ('0.bias', tensor([1.0099]))])

### Saving model

In [54]:
checkpoint = {
    'epoch' : n_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict':optimizer.state_dict(),
    'loss':losses,
    'val_loss':val_losses
}

torch.save(checkpoint,'model_checkpoint.pth')

loading model

In [55]:
%run -i data_preparation/v2.py
%run -i model_configuration/v3.py

In [56]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[0.7645]])), ('0.bias', tensor([0.8300]))])


In [57]:
checkpoint = torch.load('model_checkpoint.pth')

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

saved_epoch = checkpoint['epoch']
saved_losses = checkpoint['loss']
saved_val_losses = checkpoint['val_loss']

model.train() # use train if you want to resume training

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
)

In [58]:
model.state_dict()

OrderedDict([('0.weight', tensor([[1.9583]])), ('0.bias', tensor([1.0099]))])

In [59]:
%run -i model_training/v5.py

In [60]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9583]])), ('0.bias', tensor([1.0099]))])


loading for inference

In [61]:
%run -i model_configuration/v3.py

In [62]:
checkpoint = torch.load('model_checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9583]])), ('0.bias', tensor([1.0099]))])


In [63]:
new_inputs = torch.tensor([[.20], [.34], [.57]])
model.eval() # always use EVAL for fully trained models!
model(new_inputs.to(device))

tensor([[1.4015],
        [1.6757],
        [2.1261]], grad_fn=<AddmmBackward0>)